<a href="https://colab.research.google.com/github/satyam26en/TIME_TABLE_KIIT/blob/main/FINAL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd

# Load the section, professional elective, and core section files from GitHub
section_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/SECTION.csv'
elective_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/Elective_TIME_TABLE.csv'
core_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/CORE_TIME_TABLE_2%20-%20Sheet1.csv'

section_df = pd.read_csv(section_url)
elective_df = pd.read_csv(elective_url)
core_df = pd.read_csv(core_url)

# Normalize the 'Roll No.' column to ensure there are no leading/trailing spaces and consistent data type
section_df['Roll No.'] = section_df['Roll No.'].astype(str).str.strip()

# Define the order of days and times
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
times = ['8 TO 9', '9 TO 10', '10 TO 11', '11 TO 12', '12 TO 1', '1 TO 2', '2 TO 3', '3 TO 4', '4 TO 5']

# Define a mapping from abbreviated to full day names
day_mapping = {
    'MON': 'Monday',
    'TUE': 'Tuesday',
    'WED': 'Wednesday',
    'THU': 'Thursday',
    'FRI': 'Friday',
    'SAT': 'Saturday'
}

# Function to standardize time slot names
def standardize_time_slot(time_slot):
    time_slot_mapping = {
        '8 TO 9': '8 TO 9',
        '9 TO 10': '9 TO 10',
        '10 TO 11': '10 TO 11',
        '11 TO 12': '11 TO 12',
        '12 TO 1': '12 TO 1',
        '1 TO 2': '1 TO 2',
        '2 TO 3': '2 TO 3',
        '3 TO 4': '3 TO 4',
        '4 TO 5': '4 TO 5'
    }
    # Remove extra spaces in the time slot string to match the mapping
    standardized_slot = ' '.join(time_slot.upper().split())
    return time_slot_mapping.get(standardized_slot, time_slot)

# Function to take roll number input and display the timetable
def display_timetable():
    # Take roll number input from the user
    roll_number = input("Enter Roll Number: ").strip()

    # Find the section details for the given roll number
    student_section = section_df[section_df['Roll No.'] == roll_number]

    if student_section.empty:
        print("Roll number not found.")
        return

    # Extract the core section and elective sections
    core_section = student_section['Core Section'].values[0]
    elective_1_section = student_section['Professional Elective 1'].values[0]
    elective_2_section = student_section['Professional Elective 2'].values[0]

    # Retrieve the weekly timetable for Professional Electives 1 and 2
    elective_1_timetable = elective_df[elective_df['Section(DE)'] == elective_1_section]
    elective_2_timetable = elective_df[elective_df['Section(DE)'] == elective_2_section]
    core_timetable = core_df[core_df['Section'] == core_section]

    # Initialize the timetable matrix
    timetable_matrix = pd.DataFrame(index=times, columns=days, data='')

    # Function to fill the timetable matrix with subject names and room numbers
    def fill_timetable(timetable_df):
        room_columns = [col for col in timetable_df.columns if 'ROOM' in col]
        for index, row in timetable_df.iterrows():
            day = day_mapping.get(row['DAY'], 'Unknown')
            if day == 'Unknown':
                continue  # Skip if day is not in the mapping
            for col in room_columns:
                if row[col] != '---':
                    time_col = timetable_df.columns[timetable_df.columns.get_loc(col) + 1]
                    time_slot = standardize_time_slot(time_col)
                    subject = row.get(time_col, 'N/A')
                    room_number = row[col]
                    if subject.lower() != 'x':  # Only include if it's not 'x'
                        timetable_matrix.at[time_slot, day] = f"{subject} ({room_number})"

    # Fill the timetable matrix for core and elective timetables
    fill_timetable(core_timetable)
    fill_timetable(elective_1_timetable)
    fill_timetable(elective_2_timetable)

    # Replace NaN values with blank spaces
    timetable_matrix = timetable_matrix.fillna('')

    # Sort the table based on time slots
    timetable_matrix = timetable_matrix.reindex(times)

    # Display the filled timetable matrix
    print("\nCombined Timetable:")
    print(timetable_matrix.to_string())

    # Save the DataFrame as an Excel file
    timetable_matrix.to_excel('timetable.xlsx', index=True)

# Run the function
display_timetable()


Enter Roll Number: 22052171

Combined Timetable:
                   Monday           Tuesday         Wednesday         Thursday            Friday        Saturday
8 TO 9                                         DAA (A-LH-107)       CN (B-402)                                  
9 TO 10                                         CN (A-LH-107)  DAA(L) (WL-104)                                  
10 TO 11    EE (A-LH-107)                       SE (A-LH-107)  DAA(L) (WL-104)     EE (A-LH-207)                
11 TO 12    SE (A-LH-004)                    DMDW(DE) (B-306)    SE (A-LH-005)                       SE (WL-103)
12 TO 1    DAA (A-LH-004)                     DOS(DE) (B-404)   DAA (A-LH-005)                    CN(L) (WL-103)
1 TO 2      CN (A-LH-004)                                        EE (A-LH-005)  DMDW(DE) (B-306)  CN(L) (WL-103)
2 TO 3                                                                                                          
3 TO 4    DOS(DE) (B-404)                      